# Import Lib

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from random import randint
from scipy.io import mmread
from community import community_louvain

# Import Graph

In [ ]:

read_file = mmread('power-bcspwr10.mtx')
G_init = nx.Graph(read_file)

Try new visualization

In [ ]:
# Setting position of nodes for multiple visualizations
pos = nx.spring_layout(G_init, iterations=500, seed=1337) # Change iterations on slower hardware

In [ ]:
# selfloops version
fig, ax = plt.subplots(figsize=(15, 9))
ax.axis("off")
plot_options = {"node_size": 10, "with_labels": False, "width": 0.15}
nx.draw_networkx(G_init, pos=pos, ax=ax, **plot_options)
G = G_init.copy()

In [ ]:

# no selfloops version
G.remove_edges_from(nx.selfloop_edges(G))
fig, ax = plt.subplots(figsize=(15, 9))
ax.axis("off")
plot_options = {"node_size": 10, "with_labels": False, "width": 0.15}
nx.draw_networkx(G, pos=pos, ax=ax, **plot_options)

# Basic topological attributes

In [ ]:
def measure(G):
    print(f"Number of nodes: {G.number_of_nodes()}")
    print(f"Number of edges: {G.number_of_edges()}")
    print(f"Density: {nx.density(G)}")
    print(f"Transitivity: {nx.transitivity(G)}")
    print(f"Average Clustering: {nx.average_clustering(G)}")
    print(f"Max Degree: {max(dict(G.degree()).values())}")
    print(f"Min Degree: {min(dict(G.degree()).values())}")
    print(f"Average Degree: {np.mean(list(dict(G.degree()).values()))}")
    print(f"Number of components:{nx.number_connected_components(G)}")
    print(f"Assortativity: {nx.degree_assortativity_coefficient(G)}")

    if(nx.is_connected(G)):
        print("Graph is connected")
        print(f"Diameter: {nx.diameter(G)}")
    else:
        print("Graph is not connected")

In [ ]:
measure(G)

In [ ]:
betwenness_centrality = nx.betweenness_centrality(G)
closeness_centrality = nx.closeness_centrality(G)
degree_centrality = nx.degree_centrality(G)

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(131)
plt.title("Betweenness distribution")
plt.hist(betwenness_centrality.values(), bins=100, log=True)

plt.subplot(132)
plt.title("Closeness distribution")
plt.hist(closeness_centrality.values(), bins=100)

plt.subplot(133)
plt.title("Degree centrality distribution")
plt.hist(degree_centrality.values())

plt.show()

In [ ]:
print(f"Betweenness centrality max:{max(betwenness_centrality.values())}")
print(f"Closeness centrality max:{max(closeness_centrality.values())}")
print(f"Degree centrality max:{max(degree_centrality.values())}")

In [ ]:
#number of unique triangle in the network
triangles_per_node = list(nx.triangles(G).values())
sum(
    triangles_per_node
) / 3  # divide by 3 because each triangle is counted once for each node

# Communities

In [ ]:
partition = community_louvain.best_partition(G)
# assign a unique color to each community
colors = [partition[n] for n in G.nodes()]

In [ ]:
pos = nx.spring_layout(G, iterations=500, seed=1337)
plt.figure(figsize=(15, 8))
# draw the graph with nodes colored by community
nx.draw_networkx_nodes(G, pos, node_color=colors, cmap=plt.cm.Set1, node_size=10)
nx.draw_networkx_edges(G, pos)


In [ ]:
# print number of comm
print(f"Number of communities: {len(set(partition.values()))}")

In [ ]:
degree_sequence = sorted((d for n, d in G.degree()), reverse=True)
dmax = max(degree_sequence)
plt.bar(*np.unique(degree_sequence, return_counts=True))
#average degree vicino  alla mediana: 7 -> non è scale free

fig.tight_layout()
plt.title("Communities distribution ")
plt.xlabel("Size of communities")
plt.ylabel("Counts")
plt.show()

## Generating random graph

In [ ]:
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()
print(f"Number of nodes: {num_nodes}")
print(f"Number of edges: {num_edges}")
RG = nx.gnm_random_graph(num_nodes, num_edges, seed=1337)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 9))
ax.axis("off")
plot_options = {"node_size": 10, "with_labels": False, "width": 0.15}
nx.draw_networkx(RG, pos=nx.spring_layout(RG, iterations=500, seed=1337), ax=ax, **plot_options)

We just Analize the Biggest component

In [ ]:
largest_comp = max(nx.connected_components(RG), key=len)
RGL = RG.subgraph(largest_comp)

pos_random = nx.spring_layout(RGL, iterations=100, seed=1337) # Change iterations on slower hardware
fig, ax = plt.subplots(figsize=(15, 9))
ax.axis("off")
plot_options = {"node_size": 10, "with_labels": False, "width": 0.15}
nx.draw_networkx(RGL, pos=pos_random, ax=ax, **plot_options)

compute measures

In [ ]:
measure(RGL)

Average Degree Distribution

In [ ]:
degree_sequence = sorted((d for n, d in RGL.degree()), reverse=True)
dmax = max(degree_sequence)
plt.bar(*np.unique(degree_sequence, return_counts=True))
#average degree vicino  alla mediana: 7 -> non è scale free

fig.tight_layout()
plt.show()

Betwenness,Closeness and Degree Centrality

In [ ]:
betwenness_centrality = nx.betweenness_centrality(RGL)
closeness_centrality = nx.closeness_centrality(RGL)
degree_centrality = nx.degree_centrality(RGL)

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(131)
plt.title("Betweenness distribution")
plt.hist(betwenness_centrality.values(), bins=100, log=True)

plt.subplot(132)
plt.title("Closeness distribution")
plt.hist(closeness_centrality.values(), bins=100)

plt.subplot(133)
plt.title("Degree centrality distribution")
plt.hist(degree_centrality.values())#, bins=100)

plt.show()

In [ ]:
print(f"Betweenness centrality max:{max(betwenness_centrality.values())}")
print(f"Closeness centrality max:{max(closeness_centrality.values())}")
print(f"Degree centrality max:{max(degree_centrality.values())}")

# Random Graph Communities

In [ ]:
partition = community_louvain.best_partition(RGL)
# assign a unique color to each community
colors = [partition[n] for n in RGL.nodes()]

In [ ]:
pos = nx.spring_layout(RGL, iterations=500, seed=1337)
plt.figure(figsize=(15, 8))
# draw the graph with nodes colored by community
nx.draw_networkx_nodes(RGL, pos, node_color=colors, cmap=plt.cm.Set1, node_size=10)
nx.draw_networkx_edges(RGL, pos)


In [ ]:
# print number of comm
print(f"Number of communities: {len(set(partition.values()))}")

N° of triangles

In [ ]:
#number of unique triangle in the network
triangles_per_node = list(nx.triangles(RGL).values())
sum(
    triangles_per_node
) / 3  # divide by 3 because each triangle is counted once for each node